In [129]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

"""

Función para segmentación por el método de Watersheed

La segmentación por Watersheed es un algoritmo basado en la topografía de la imagen,
donde las regiones de interés se consideran como "cuencas" que se llenan de agua desde los "mínimos" (regiones oscuras) hasta que se encuentran con otras cuencas,
formando así los bordes de las regiones segmentadas.

La función `segmentacion_watersheed` toma una imagen en escala de grises y un número de iteraciones para las operaciones morfológicas.

La funcion se utiliza con el proposito de obtener o resaltar los bordes de la estrutura que se encuentra en el primer plano de la imagen.

Args:
    imagen (numpy.ndarray): Imagen en escala de grises a segmentar.
    iteraciones (int): Número de iteraciones para las operaciones morfológicas.

"""

def segmentacion_watersheed(imagen, iteraciones):
    
    # Primero, necesitamos encontrar los marcadores para las regiones
    _, thresh = cv.threshold(imagen, 0, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)

    # Definir kernel para operaciones morfológicas
    kernel = np.ones((3, 3), dtype=np.uint8)
    opening = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel, iterations=iteraciones)
   
    sure_bg = cv.erode(opening, kernel, iterations=iteraciones)

    dist_transform = cv.distanceTransform(thresh, cv.DIST_L2, 5)

    _, sure_fg = cv.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

    sure_fg = np.uint8(sure_fg)
    unknown = cv.subtract(sure_bg, sure_fg)

    #Marcadores
    _, markers = cv.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0

    # Aplicar el algoritmo de watershed
    markers = cv.watershed(cv.cvtColor(sure_bg, cv.COLOR_GRAY2BGR), markers)
    
    imagen[markers == -1] = 255  # Marcar los bordes con blanco

    return imagen

In [130]:
import cv2 as cv
import numpy as np
from collections import deque

"""

Función para segmentación por el método de Semilla

La segmentación por semilla es un algoritmo que comienza con uno o más píxeles "semilla" y expande la región alrededor de estas semillas
basándose en un criterio de similitud, como la intensidad de píxeles.

La función `segmentacion_semilla` toma una lista de semillas, un umbral de similitud, una imagen en escala de grises y la conectividad (4 o 8).

La funcion apunto a resaltar la estructura que se encuentra en el primer plano de la imagen.

Args:
    seeds (Lista de tuplas): Lista de coordenadas (x, y) de las semillas.
    threshold (int): Umbral de similitud para incluir píxeles en la región segmentada.
    imagen (numpy.ndarray): Imagen en escala de grises a segmentar.
    conectividad (int): Tipo de conectividad (4 o 8) para definir vecinos.

"""

def segmentacion_semilla(seeds, threshold, imagen, conectividad):
    segmented = np.zeros_like(imagen, dtype=np.uint8) #arreglo de 0's para la segmentacion (negro)
    visited = np.zeros_like(imagen, dtype=bool) #arreglo de booleanos para marcar los pixeles visitados
    height, width = imagen.shape # dimensiones de la imagen

    # Vecinos según conectividad
    if conectividad == 4:
        vecinos = [(-1,0), (1,0), (0,-1), (0,1)]
    else:
        vecinos = [(dx,dy) for dx in [-1,0,1] for dy in [-1,0,1] if not (dx==0 and dy==0)]

    for seed in seeds:
        x, y = seed

        if not (0 <= x < height and 0 <= y < width):
            print(f"Semilla {seed} fuera de límites (height={height}, width={width}), se omite.")
            continue

        if visited[x, y]:
            continue

        queue = deque()
        queue.append((x, y))
        visited[x, y] = True    # marcar pixel como visitado
        segmented[x, y] = 255   # marcar pixel en la segmentacion blanco
        region_sum = int(imagen[x, y])
        region_count = 1
        while queue:
            px, py = queue.popleft()
            region_mean = region_sum // region_count
            for dx, dy in vecinos:
                nx, ny = px + dx, py + dy
                if 0 <= nx < height and 0 <= ny < width and not visited[nx, ny]:
                    if abs(int(imagen[nx, ny]) - region_mean) < threshold:
                        segmented[nx, ny] = 255
                        visited[nx, ny] = True
                        queue.append((nx, ny))
                        region_sum += int(imagen[nx, ny])
                        region_count += 1
    return segmented


In [131]:
import cv2 as cv

def desplegar_imagenes(watershed_img, seeded_img):
    # Mostrar resultados de segmentación
    cv.imshow('Segmentacion por Watershed', watershed_img)
    cv.imshow('Segmentacion por semilla', seeded_img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [132]:
import cv2 as cv
import os
import sys

"imagen de 194 x 259 pixeles)"
def segmentar_imagen_monterrey():
    # Cargar la imagen
    raiz = os.getcwd()
    imgPath = os.path.join(raiz, 'monterrey.jpg')
    img = cv.imread(imgPath)
    gris = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    if img is None:
        print('Error al cargar la imagen')
        sys.exit(1)
    
    seeds = [(25, 25), (100, 100), (125, 125), (200, 200)]  # ajustar semilla(s) y umbral según convenga
    threshold = 45
    iteraciones = 3
    conectividad = 4
    res_watersheed = segmentacion_watersheed(gris, iteraciones)

    res_seeded = segmentacion_semilla(seeds, threshold, gris, conectividad)

    desplegar_imagenes(res_watersheed, res_seeded)

In [133]:
import cv2 as cv
import os
import sys

"imagen de 183 x 275 pixeles)"
def segmentar_imagen_sujeto():
    # Cargar la imagen
    raiz = os.getcwd()
    imgPath = os.path.join(raiz, 'sujeto.jpg')
    img = cv.imread(imgPath)
    gris = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    if img is None:
        print('Error al cargar la imagen')
        sys.exit(1)
    
    #seeds = [(25, 25), (50, 50), (0, 100), (0, 125), (25, 125), (80, 125), (125,80)]  # ajustar semilla(s) y umbral según convenga
    seeds = [(25, 50), (50, 100)]
    threshold = 35
    iteraciones = 5
    conectividad = 4
    res_watersheed = segmentacion_watersheed(gris, iteraciones)

    res_seeded = segmentacion_semilla(seeds, threshold, gris, conectividad)

    desplegar_imagenes(res_watersheed, res_seeded)

In [134]:
import cv2 as cv
import os
import sys

"imagen de 148 x 163 pixeles"
def segmentar_imagen_lena():
    # Cargar la imagen
    raiz = os.getcwd()
    imgPath = os.path.join(raiz, 'lena.jpg')
    img = cv.imread(imgPath)
    gris = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    if img is None:
        print('Error al cargar la imagen')
        sys.exit(1)
    
    seeds = [(25, 25), (125, 25), (80, 160)]  # ajustar semilla(s) y umbral según convenga
    threshold = 45
    iteraciones = 4
    conectividad = 8
    res_watersheed = segmentacion_watersheed(gris, iteraciones)

    res_seeded = segmentacion_semilla(seeds, threshold, gris, conectividad)

    desplegar_imagenes(res_watersheed, res_seeded)

In [135]:
def mostrar_imagen_segmentada(*imagenes):
    for imagen in imagenes:
        if imagen == 'monterrey':
            segmentar_imagen_monterrey()
        elif imagen == 'sujeto':
            segmentar_imagen_sujeto()
        elif imagen == 'lena':
            segmentar_imagen_lena()

In [136]:
"""
Ejercicios de segmentacion usando diferentes metodos: 
    Watersheed
    Seeds
    Split and Merge.

-Se trata de segmentar tres imagenes diferentes: monterrey.jpg, sujeto.jpg y lena.jpg esto con la finalidad de resaltar las estructuras al frente de un fondo complejo.
-Se pueden cambiar las semillas y los umbrales para observar diferentes resultados.
-Las imagenes deben estar en el mismo directorio que este script.
-Cada metodo de segmentacion se visualiza en una ventana diferente.
-Cada vez que se llama a la funcion mostrar_imagen_segmentada con el nombre de la imagen, se ejecuta el proceso de segmentacion para esa imagen.
    el metodo se puede llamar haciendo referencia a las 3 imagenes, lo que mostrara las segmentaciones de las 3 imagenes en ventanas separadas.
-Tras analizar los resultados obtenidos se determina que los segmentos mas definidos se obtienen con el metodo de Watersheed, seguido del metodo de semillas y por ultimo el metodo de Split and Merge.

"""
mostrar_imagen_segmentada('lena')